In [2]:
import tensorflow as tf
from keras import layers
from keras import preprocessing
import word2vec as w2v
import dataProsessing as dp
import numpy as np
from tensorboard.plugins import projector
import os
import io
import json

In [3]:

#run this once and only once
os.chdir("../")


In [4]:
datasetName = "Jsb16thSeparated(t_60_rr_re)"

corpus = dp.loadDataset("Data\\Corpi\\" + datasetName + "\\" + datasetName +".json")
vocab = dataset = json.load(open("Data\\Corpi\\" + datasetName + "\\" + datasetName +"_vocab.json"))
inv_vocab = vocab["inv_vocab"]
vocab = vocab["vocab"]



In [5]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
embedding_dim = 128
negative_samples = 15
log_dir = "logs\embeddings"
model_name =  "d_" + str(embedding_dim) + " n_" + str(negative_samples)

vectorizedSequences = dp.vectorizeDataset(corpus,vocab)
targets,contexts,labels = [],[],[]
for s in vectorizedSequences:
    tmp = w2v.generateTrainingData(s,vocab_size=len(vocab),window_size=1,negative_samples=negative_samples)
    targets += tmp[0]
    contexts += tmp[1]
    labels += tmp[2]
    
targets = np.array(targets)
contexts = np.array(contexts)
labels = np.array(labels)

dataset = tf.data.Dataset.from_tensor_slices(((targets,contexts),labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


word2vec = w2v.Word2Vec(len(vocab), embedding_dim,negative_samples)
word2vec.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")
word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])




Epoch 1/20
64/64 [==============================] - 4s 50ms/step - loss: 2.7590 - accuracy: 0.2841
Epoch 2/20
64/64 [==============================] - 3s 50ms/step - loss: 2.6584 - accuracy: 0.7914
Epoch 3/20
64/64 [==============================] - 3s 51ms/step - loss: 2.3798 - accuracy: 0.8720
Epoch 4/20
64/64 [==============================] - 3s 54ms/step - loss: 1.9776 - accuracy: 0.8434
Epoch 5/20
64/64 [==============================] - 4s 57ms/step - loss: 1.6203 - accuracy: 0.8319
Epoch 6/20
64/64 [==============================] - 3s 50ms/step - loss: 1.3493 - accuracy: 0.8415
Epoch 7/20
64/64 [==============================] - 4s 55ms/step - loss: 1.1343 - accuracy: 0.8606
Epoch 8/20
64/64 [==============================] - 3s 52ms/step - loss: 0.9553 - accuracy: 0.8828
Epoch 9/20
64/64 [==============================] - 4s 55ms/step - loss: 0.8038 - accuracy: 0.9037
Epoch 10/20
64/64 [==============================] - 3s 51ms/step - loss: 0.6749 - accuracy: 0.9227
Epoch 11/

In [6]:

weights = tf.Variable(word2vec.get_layer('w2v_embedding').get_weights()[0])




checkpoint = tf.train.Checkpoint(embedding=weights)
checkpoint.save(os.path.join("data\\corpi\\" + datasetName + "\\tensorboard data\\" + model_name, "embedding.ckpt"))




'data\\corpi\\Jsb16thSeparated(t_60_rr_re)\\tensorboard data\\d_128 n_15\\embedding.ckpt-1'

In [7]:
weights = np.array(weights).tolist()

with open("data\\corpi\\" + datasetName + "\\weights\\" + model_name, "w") as outfile:
    json.dump(weights, outfile)


In [8]:
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`.
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings("data\\corpi\\" + datasetName  + "\\tensorboard data\\" + model_name, config)

In [9]:

%tensorboard --logdir logs\embeddings


UsageError: Line magic function `%tensorboard` not found.
